# Preparing datasets

In [ ]:
IMAGE_SIZE = [256, 256]

def decode(img):
    image = tf.image.decode_jpeg(img, channels=3)
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def normalize(img):
    return (tf.cast(img, tf.float32) / 127.5) - 1

def flip(img):
    return tf.image.flip_left_right(img)

def random_crop(img):
    cropped_image = tf.image.random_crop(img, size=[256, 256, 3])
    return cropped_image

def random_jitter(img):
    image = tf.image.resize(img, [int(256*1.3), int(256*1.3)],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    image = random_crop(image)
    return image

def preprocess_image_train(img, label=None):
    image = random_jitter(img)
    return image

def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode(example['image'])
    return image

def load_dataset(filenames, labeled=False, ordered=False, repeats=200):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset = dataset.concatenate(dataset.map(flip, num_parallel_calls=AUTOTUNE).shuffle(100000))
    dataset = dataset.concatenate(dataset.map(random_jitter, num_parallel_calls=AUTOTUNE).shuffle(10000, reshuffle_each_iteration=True).repeat(repeats))
    dataset = dataset.map(normalize, num_parallel_calls=AUTOTUNE).shuffle(10000)
    return dataset

In [ ]:
monet_ds = load_dataset(MONET_FILENAMES, labeled=True, repeats=50).batch(100, drop_remainder=True)
photo_ds = load_dataset(PHOTO_FILENAMES, labeled=True, repeats=2  ).batch(100, drop_remainder=True)

# Upsamplin and Downsampling

In [ ]:
OUTPUT_CHANNELS = 3
LATENT_DIM = 1024

def downsampling(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    r = keras.Sequential()
    r.add(layers.Conv2D(filters, size, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    r.add(layers.MaxPool2D())

    if apply_instancenorm:
        r.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    r.add(layers.LeakyReLU())

    return r


In [ ]:
def upsampling(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    r = keras.Sequential()
    r.add(layers.Conv2DTranspose(filters, size, strides=2,
                                      padding='same',
                                      kernel_initializer=initializer,
                                      use_bias=False))

    result.add(tfa.layers.InstanceNormalization(gamma_initializer=gamma_init))

    if apply_dropout:
        r.add(layers.Dropout(0.5))

    r.add(layers.LeakyReLU())

    return result


In [ ]:
EPOCHS = 25

LR_G = 2e-4
LR_D = 2e-4
beta_1 = .5

RLabel = .9
Flabel = 0

# Generator

In [ ]:
def Generator():
    inputs = layers.Input(shape=[256,256,3])

    downstack = [
        downsampling(64, 4, apply_instancenorm=False),
        downsampling(128, 4), 
        downsampling(256, 4), 
        downsampling(512, 4), 
        downsampling(512, 4), 
        downsampling(512, 4), 
        downsampling(512, 4), 
        downsampling(512, 4), 
    ]

    upstack = [
        upsampling(512, 4, apply_dropout=True), 
        upsampling(512, 4, apply_dropout=True), 
        upsampling(512, 4, apply_dropout=True), 
        upsampling(512, 4), 
        upsampling(256, 4), 
        upsampling(128, 4), 
        upsampling(64, 4), 
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                  strides=2,
                                  padding='same',
                                  kernel_initializer=initializer,
                                  activation='tanh') # (bs, 256, 256, 3)

    x = inputs

    s = []
    for d in downstack:
        x = d(x)
        s.append(x)

    s = reversed(s[:-1])

    # Upsampling and establishing the skip connections
    for up, i in zip(upstack, s):
        x = up(x)
        x = layers.Concatenate()([x, i])

    x = last(x)

    return keras.Model(inputs=inputs, outputs=x)


# Discriminator

In [ ]:
def discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    inp = layers.Input(shape=[256, 256, 3], name='input_image')

    x = inp

    d1 = downsampling(64, 4, False)(x) 
    d2 = downsampling(128, 4)(d1)
    d3 = downsampling(256, 4)(d2) 

    zero_pad1 = layers.ZeroPadding2D()(3) 
    conv = layers.Conv2D(512, 4, strides=1,
                         kernel_initializer=initializer,
                         use_bias=False)(zero_pad1) 

    norm1 = tfa.layers.InstanceNormalization(gamma_initializer=gamma_init)(conv)

    leaky_relu = layers.LeakyReLU()(norm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu) 

    last_conv = layers.Conv2D(1, 4, strides=1,
                         kernel_initializer=initializer)(zero_pad2) # (bs, 30, 30, 1)

    last_relu = layers.LeakyReLU(alpha=0.2)(last_conv)
    last_pool = layers.Flatten()(last_relu)
    last = layers.Dense(1, activation='sigmoid')(last_pool)

    return tf.keras.Model(inputs=inp, outputs=last)

In [ ]:
monet_CycleGenerator = generator() 
monet_CycleDiscriminator = discriminator()

photo_CycleGenerator = generator() 
photo_CycleDiscriminator = discriminator() 

In [ ]:
class CycleGan(keras.Model):
    def __init__(self,monetG,photoG,monetD,photoD,lambda_cycle=10,Rlabel=.5):
        super(CycleGan, self).__init__()
        self.m_gen = monetG
        self.p_gen = photoG
        self.m_disc = monetD
        self.p_disc = photoD
        self.lambda_cycle = lambda_cycle
        self.Rlabel = Rlabel
        
    def compile(self,m_gen_optimizer,p_gen_optimizer,m_disc_optimizer,p_disc_optimizer,gen_loss_fn,disc_loss_fn,cycle_loss_fn,identity_loss_fn):
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        
    def train_step(self, batch_data):
        realMonet, realPhoto = batch_data
        
        batch_size = tf.shape(realPphoto)[0]
        labels_real = tf.zeros((batch_size, 1)) + self.Rlabel
        labels_real += 0.05 * tf.random.uniform(tf.shape(labels_real))        
        
        with tf.GradientTape(persistent=True) as tape:
            fakeMonet = self.m_gen(reaPhoto, training=True)
            cycled_photo = self.p_gen(fakeMonet, training=True)

            
            fakePhoto = self.p_gen(realMonet, training=True)
            cycledMonet = self.m_gen(fakePhoto, training=True)

            
            same_monet = self.m_gen(realMonet, training=True)
            same_photo = self.p_gen(realPhoto, training=True)

            
            disc_real_monet = self.m_disc(realMonet, training=True)
            disc_real_photo = self.p_disc(realPhoto, training=True)

            
            disc_fake_monet = self.m_disc(fakeMonet, training=True)
            disc_fake_photo = self.p_disc(fakePhoto, training=True)

            
            monet_gen_loss = self.gen_loss_fn(disc_real_monet, disc_fake_monet, labels_real)
            photo_gen_loss = self.gen_loss_fn(disc_real_photo, disc_fake_photo, labels_real)

            
            total_cycle_loss = self.cycle_loss_fn(realMonet, cycled_monet, self.lambda_cycle) + self.cycle_loss_fn(realPhoto, cycled_photo, self.lambda_cycle)

            
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + self.identity_loss_fn(realMonet, same_monet, self.lambda_cycle)
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + self.identity_loss_fn(realPhoto, same_photo, self.lambda_cycle)

            
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet, labels_real)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo, labels_real)

        
        monet_generator_gradients = tape.gradient(total_monet_gen_loss,
                                                  self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss,
                                                  self.p_gen.trainable_variables)

        monet_discriminator_gradients = tape.gradient(monet_disc_loss,
                                                      self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss,
                                                      self.p_disc.trainable_variables)

        
        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients,
                                                 self.m_gen.trainable_variables))

        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients,
                                                 self.p_gen.trainable_variables))

        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients,
                                                  self.m_disc.trainable_variables))

        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients,
                                                  self.p_disc.trainable_variables))
        
        return {
            "monet_gen_loss": total_monet_gen_loss,
            "photo_gen_loss": total_photo_gen_loss,
            "monet_disc_loss": monet_disc_loss,
            "photo_disc_loss": photo_disc_loss
        }

In [ ]:
def discriminator_loss(predictions_real, predictions_gen, labels_real):
        return (tf.reduce_mean((predictions_gen  - tf.reduce_mean(predictions_real) + labels_real) ** 2) +
                tf.reduce_mean((predictions_real - tf.reduce_mean(predictions_gen)  - labels_real) ** 2))/2
    
def generator_loss(predictions_real, predictions_gen, labels_real):
        return (tf.reduce_mean((predictions_real - tf.reduce_mean(predictions_gen)  + labels_real) ** 2) +
                tf.reduce_mean((predictions_gen  - tf.reduce_mean(predictions_real) - labels_real) ** 2)) / 2

In [ ]:
def calc_cycle_loss(real_image, cycled_image, LAMBDA):
        loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))

        return LAMBDA * loss1

In [ ]:
with x:
    def identity_loss(real_image, same_image, LAMBDA):
        loss = tf.reduce_mean(tf.abs(real_image - same_image))
        return LAMBDA * 0.5 * loss

In [ ]:
monet_generator_optimizer = tf.keras.optimizers.Adam(LR_G, beta_1=0.5)
monet_discriminator_optimizer = tf.keras.optimizers.Adam(LR_D, beta_1=0.5)

photo_generator_optimizer = tf.keras.optimizers.Adam(LR_G, beta_1=0.5)
photo_discriminator_optimizer = tf.keras.optimizers.Adam(LR_D, beta_1=0.5)

In [ ]:
cycle_gan_model = CycleGan(
        monet_cycleGenerator, photo_cycleGenerator, monet_cycleDiscriminator, photo_cycleDiscriminator, label=0.66
)

cycle_gan_model.compile(
        m_gen_optimizer = monet_generator_optimizer,
        p_gen_optimizer = photo_generator_optimizer,
        m_disc_optimizer = monet_discriminator_optimizer,
        p_disc_optimizer = photo_discriminator_optimizer,
        gen_loss_fn = generator_loss,
        disc_loss_fn = discriminator_loss,
        cycle_loss_fn = calc_cycle_loss,
        identity_loss_fn = identity_loss
)

In [ ]:
cycle_gan_model.fit(
    tf.data.Dataset.zip((monet_ds, photo_ds)),
    epochs=EPOCHS
)

# Visualization

In [ ]:
def view_image(ds, rows=2):
    image = next(iter(ds))
    image = image.numpy()

    fig = plt.figure(figsize=(22, rows * 5 ))
    for i in range(5 * rows):
        ax = fig.add_subplot(rows, 5, i+1, xticks=[], yticks=[])
        ax.imshow(image[i] / 2 + .5)

In [ ]:
_, ax = plt.subplots(2, 5, figsize=(12, 6))
for i, img in enumerate(photo_ds.take(5)):
    generated = monet_cycleGenerator(img, training=False)[0].numpy()
    generated = (generated * 127.5 + 127.5).astype(np.uint8)
    img = (img[0] * 127.5 + 127.5).numpy().astype(np.uint8)

    ax[0, i].imshow(img)
    ax[1, i].imshow(generated)
    ax[0, i].set_title("Input")
    ax[1, i].set_title("Output")
    ax[0, i].axis("off")
    ax[1, i].axis("off")
plt.show()